In [1]:
import nltk
import string
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from multiprocessing import Pool, TimeoutError

In [2]:
# Read in the data
reviews = pd.read_csv('Reviews.csv')
reviews = reviews.sample(frac=0.2)

In [3]:
# Clean the data
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
reviews_without_stopwords = [word for word in reviews['Text'].str.split() if word not in stop_words]

In [4]:
punctuation = list(string.punctuation)

In [22]:
# Set everything to lower case and remove punctuation
reviews_without_stopwords_or_punctuation = []
# reviews_without_stopwords_or_punctuation_full_list = []
for st in reviews_without_stopwords:
	tmp = []
	for word in st:
		if word not in punctuation:
			tmp.append(word.lower())
			# reviews_without_stopwords_or_punctuation_full_list.append(word.lower())
	reviews_without_stopwords_or_punctuation.append(tmp)

In [23]:
# print(len(reviews_without_stopwords_or_punctuation))
print(reviews_without_stopwords_or_punctuation[0])
print(len(reviews_without_stopwords_or_punctuation))
# print(reviews['Score'])

["amazon's", 'subscribe', 'and', 'save', 'program', 'has', 'the', 'best', 'deal', 'on', 'these', 'that', 'i', 'have', 'found', 'so', 'far.', 'these', 'taste', 'very', 'good', 'and', 'i', 'feel', 'good', 'about', 'my', 'kids', 'snacking', 'on', 'these', 'when', 'we', 'are', 'out', 'and', 'about.', 'they', 'are', 'so', 'easy', 'to', 'toss', 'in', 'your', 'purse.', 'i', 'do', 'not', 'keep', 'candy', 'around', 'so', 'my', 'kids', 'view', 'these', 'as', 'their', 'sweet', 'treat.']
113691


In [31]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


def lemmatize_string(word_list):
	return ' '.join([lemmatizer.lemmatize(words) for words in word_list])

with Pool(processes=8) as pool:
	reviews_without_stopwords_or_punctuation_lemmatize = pool.map(lemmatize_string, reviews_without_stopwords_or_punctuation)

In [32]:
reviews_without_stopwords_or_punctuation_lemmatize[1]

'i love the flavor of sliver of chewy coconut, honey and almond it really doe taste great to me.so far this is my favorite kind'

In [33]:
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True, stop_words=None,
                     ngram_range=(1, 1), tokenizer=token.tokenize)
reviews_tokenized = cv.fit_transform(reviews_without_stopwords_or_punctuation_lemmatize)

In [34]:
print(reviews_tokenized.shape)

(113691, 59126)


In [35]:
review_score = [int(i) for i in reviews['Score']]

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reviews_tokenized, review_score, test_size=0.2, random_state=0)

In [37]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
clf = DecisionTreeClassifier().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("Decision Tree Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))

Decision Tree Accuracy: 0.6695545098729055
Precision: 0.4858570822144527
Recall: 0.4634099440996378


In [ ]:
from sklearn.svm import SVC
clf = SVC().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("SVM Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Precision: " + str(metrics.precision_score(y_test, predicted, average='macro')))
print("Recall: " + str(metrics.recall_score(y_test, predicted, average='macro')))